# Data abstraction & Type abstraction

## Agenda

- Essentials concepts
    - Type
    - Type System
    - Polymorphism
- Python Object
- Python Type System
- Python Class

## Type

### Why type?

- Types <em style="color: red">provide</em> <em style="color: blue">implicit context</em> for many operations, so that the programmer does not have to specify that context explicitly.

- Types <em style="color: red">limit</em> <em style="color: blue">the set of operations</em> that may be performed in a semantically valid program.

- If types are specified explicitly in the source program, they can often make the program easier to read and understand.

- If types are known at compile time, they can be used to drive important performance optimizations.

## Type System

Informally, a type system consists of
1. a mechanism to <em style="color: blue">define types</em> and associate them with certain language constructs, and
2. <em style="color: red">a set of rules</em> for <em style="color: blue">type equivalence, type compatibility, and type inference.</em>

Type compatibility is the one of most concern to programmers. In many languages, however, compatibility is a looser relationship than equivalence: objects and contexts are often compatible even when their types are different.

#### Type equivalence
There are two principal ways of defining type equivalence.
- <em style="color: blue">Structural equivalence</em> is based on the content of type definitions.
- <em style="color: blue">Name equivalence</em> is based on the lexical occurrence of type definitions.

#### Type compatibility
Most languages do not require equivalence of types in every context. Instead, they merely say that <em style="color: blue">a value's type must be compatible with that of the context in which it appears.</em>

##### coercion
Type coercion is performed automatically and implicitly whenever a language allows a value of one type to be used in a context that expects another.

**What if the coercion can not be performed? 🤔**

### Polymorphism
Parametric Polymorphism & Subtype Polymorphism

```c++
// C++: explicit parametric polymorphism
template <typename T>
T const& Max (T const& a, T const& b) { 
    return a < b ? b:a; 
}
```

```haskell
-- haskell: implicit parametric polymorphism
let max a b = if a < b then b else a
```

```python
# Python: duck typing
def max(a, b):
    return b if a < b else a
```

```c++
// C++: subtype polymorphism
BaseClass const& Max (BaseClass const& a, BaseClass const& b) {
    return a < b ? b:a; 
}
```

According to different language implementations,
- **What implicit context does the type system provide? 🤔**
- **What operations does the type system limit? 🤔**

## Recap

### Everything is an object in Python.

### PyObject & PyVarObject
```c
// https://github.com/python/cpython/blob/2.7/Include/object.h#L106
typedef struct _object {
    PyObject_HEAD
} PyObject;

typedef struct {
    PyObject_VAR_HEAD
} PyVarObject;
```

### PyObject_HEAD
```c
// https://github.com/python/cpython/blob/2.7/Include/object.h#L78
/* PyObject_HEAD defines the initial segment of every PyObject. */
#define PyObject_HEAD                   \
    _PyObject_HEAD_EXTRA                \
    Py_ssize_t ob_refcnt;               \
    struct _typeobject *ob_type;
```

```c
// https://github.com/python/cpython/blob/2.7/Include/object.h#L64
#ifdef Py_TRACE_REFS
/* ... */
#else
#define _PyObject_HEAD_EXTRA
#define _PyObject_EXTRA_INIT
#endif
```

### PyObject_VAR_HEAD
```c
// https://github.com/python/cpython/blob/2.7/Include/object.h#L96
/* PyObject_VAR_HEAD defines the initial segment of all variable-size
 * container objects. */
#define PyObject_VAR_HEAD               \
    PyObject_HEAD                       \
    Py_ssize_t ob_size; /* Number of items in variable part */
```

## Object

- Objects are Python’s abstraction for data.

- Every object has an identity, a <em style="color: blue">type</em> and a value.

- isinstance
- issubclass

https://github.com/python/cpython/blob/2.7/Objects/abstract.c#L2939


## Type

### What on earth is the type?

In [2]:
x = 0b01000001
chr(x), int(x)

('A', 65)

### Comparison of the type system

|Language | Type Safety | Type Expression                                   | Type compatibility and equivalence | Type checking |
|---------|-------------|---------------------------------------------------|------------------------------------|-----------|
|Python   | Strong      |implicit (with optional explicit typing as of 3.5) | structural | dynamic |
|C | weak | explicit | nominal | static |
| Java | strong | explicit | nominal | static |

https://en.wikipedia.org/wiki/Comparison_of_programming_languages_by_type_system

### New-style and classic classes

In [9]:
class A: pass
class B: pass
a = A()
b = B()
type(a) is type(b)

True

In [4]:
class A(object): pass
class B(object): pass
a = A()
b = B()
type(a) is type(b)

False

### New-style and classic classes

- Up to Python 2.1 <em style="color: blue">the concept of class was unrelated to the concept of type</em>, and old-style classes were the only flavor available.
- New-style classes were introduced in Python 2.2 to <em style="color: blue">unify the concepts of class and type</em>.
- The major motivation for introducing new-style classes is to <em style="color: blue">provide a unified object model with a full meta-model</em>. It also has a number of practical benefits, like the ability to subclass most built-in types, or the introduction of “descriptors”, which enable computed properties.
- For compatibility reasons, classes are still old-style by default. New-style classes are created by specifying another new-style class (i.e. a type) as a parent class, or the “top-level type” object if no other parent is needed.
- Old-style classes are removed in Python 3, <em style="color: blue">leaving only new-style classes</em>.


- https://docs.python.org/2/reference/datamodel.html#new-style-and-classic-classes
- http://python-history.blogspot.sg/2010/06/inside-story-on-new-style-classes.html

### The first commit to introduce new-style classes
https://github.com/python/cpython/commit/6d6c1a35e08b95a83dbe47dbd9e6474daff00354

### PyTypeObject

```c
// https://github.com/python/cpython/blob/2.7/Include/object.h#L324
typedef struct _typeobject {
    PyObject_VAR_HEAD
    const char *tp_name; /* For printing, in format "<module>.<name>" */
    Py_ssize_t tp_basicsize, tp_itemsize; /* For allocation */

    /* Methods to implement standard operations */

    destructor tp_dealloc;
    printfunc tp_print;
    getattrfunc tp_getattr;
    setattrfunc tp_setattr;
    cmpfunc tp_compare;
    reprfunc tp_repr;

    /* Method suites for standard classes */

    PyNumberMethods *tp_as_number;
    PySequenceMethods *tp_as_sequence;
    PyMappingMethods *tp_as_mapping;

    /* More standard operations (here for binary compatibility) */

    hashfunc tp_hash;
    ternaryfunc tp_call;
    reprfunc tp_str;
    getattrofunc tp_getattro;
    setattrofunc tp_setattro;

    /* Functions to access object as input/output buffer */
    PyBufferProcs *tp_as_buffer;

    /* Flags to define presence of optional/expanded features */
    long tp_flags;

    const char *tp_doc; /* Documentation string */

    /* Assigned meaning in release 2.0 */
    /* call function for all accessible objects */
    traverseproc tp_traverse;

    /* delete references to contained objects */
    inquiry tp_clear;

    /* Assigned meaning in release 2.1 */
    /* rich comparisons */
    richcmpfunc tp_richcompare;

    /* weak reference enabler */
    Py_ssize_t tp_weaklistoffset;

    /* Added in release 2.2 */
    /* Iterators */
    getiterfunc tp_iter;
    iternextfunc tp_iternext;

    /* Attribute descriptor and subclassing stuff */
    struct PyMethodDef *tp_methods;
    struct PyMemberDef *tp_members;
    struct PyGetSetDef *tp_getset;
    struct _typeobject *tp_base;
    PyObject *tp_dict;
    descrgetfunc tp_descr_get;
    descrsetfunc tp_descr_set;
    Py_ssize_t tp_dictoffset;
    initproc tp_init;
    allocfunc tp_alloc;
    newfunc tp_new;
    freefunc tp_free; /* Low-level free-memory routine */
    inquiry tp_is_gc; /* For PyObject_IS_GC */
    PyObject *tp_bases;
    PyObject *tp_mro; /* method resolution order */
    PyObject *tp_cache;
    PyObject *tp_subclasses;
    PyObject *tp_weaklist;
    destructor tp_del;

    /* Type attribute cache version tag. Added in version 2.6 */
    unsigned int tp_version_tag;

#ifdef COUNT_ALLOCS
    /* these must be last and never explicitly initialized */
    Py_ssize_t tp_allocs;
    Py_ssize_t tp_frees;
    Py_ssize_t tp_maxalloc;
    struct _typeobject *tp_prev;
    struct _typeobject *tp_next;
#endif
} PyTypeObject;
```

### What happened when you run following code?

In [1]:
type(type)

type

![how python runs programs](images/Entity Relationship Diagram - New Page.png "How Python runs programs")

```c
// https://github.com/python/cpython/blob/2.7/Include/object.h#L324
typedef struct _typeobject {
    /* ... */
    initproc tp_init;
    newfunc tp_new;
    /* ... */
} PyTypeObject;
```

### Built-in types

#### Declaration and initialization
```c
// https://github.com/python/cpython/blob/2.7/Include/object.h#L441
PyAPI_DATA(PyTypeObject) PyType_Type; /* built-in 'type' */

// https://github.com/python/cpython/blob/2.7/Objects/typeobject.c#L2870
PyTypeObject PyType_Type = {
    PyVarObject_HEAD_INIT(&PyType_Type, 0)
    "type",                                     /* tp_name */
    sizeof(PyHeapTypeObject),                   /* tp_basicsize */
    sizeof(PyMemberDef),                        /* tp_itemsize */
    (destructor)type_dealloc,                   /* tp_dealloc */
    0,                                          /* tp_print */
    0,                                          /* tp_getattr */
    0,                                          /* tp_setattr */
    0,                                  /* tp_compare */
    (reprfunc)type_repr,                        /* tp_repr */
    0,                                          /* tp_as_number */
    0,                                          /* tp_as_sequence */
    0,                                          /* tp_as_mapping */
    (hashfunc)_Py_HashPointer,                  /* tp_hash */
    (ternaryfunc)type_call,                     /* tp_call */
    0,                                          /* tp_str */
    (getattrofunc)type_getattro,                /* tp_getattro */
    (setattrofunc)type_setattro,                /* tp_setattro */
    0,                                          /* tp_as_buffer */
    Py_TPFLAGS_DEFAULT | Py_TPFLAGS_HAVE_GC |
        Py_TPFLAGS_BASETYPE | Py_TPFLAGS_TYPE_SUBCLASS,         /* tp_flags */
    type_doc,                                   /* tp_doc */
    (traverseproc)type_traverse,                /* tp_traverse */
    (inquiry)type_clear,                        /* tp_clear */
    type_richcompare,                                           /* tp_richcompare */
    offsetof(PyTypeObject, tp_weaklist),        /* tp_weaklistoffset */
    0,                                          /* tp_iter */
    0,                                          /* tp_iternext */
    type_methods,                               /* tp_methods */
    type_members,                               /* tp_members */
    type_getsets,                               /* tp_getset */
    0,                                          /* tp_base */
    0,                                          /* tp_dict */
    0,                                          /* tp_descr_get */
    0,                                          /* tp_descr_set */
    offsetof(PyTypeObject, tp_dict),            /* tp_dictoffset */
    type_init,                                  /* tp_init */
    0,                                          /* tp_alloc */
    type_new,                                   /* tp_new */
    PyObject_GC_Del,                            /* tp_free */
    (inquiry)type_is_gc,                        /* tp_is_gc */
};
```

- https://stackoverflow.com/questions/15404256/changing-type-of-python-objects
- Builtin-methods: https://github.com/python/cpython/blob/2.7/Python/bltinmodule.c#L2615
- _PyBuiltin_Init: https://github.com/python/cpython/blob/2.7/Python/bltinmodule.c#L2678
- PyAPI_FUNC & PyAPI_DATA: https://github.com/python/cpython/blob/2.7/Include/pyport.h#L746
- PyType_Type declaration: https://github.com/python/cpython/blob/2.7/Include/object.h#L441
- PyType_Type assignment: https://github.com/python/cpython/blob/2.7/Objects/typeobject.c#L2870
- classmethod & staticmethod: https://github.com/python/cpython/blob/2.7/Objects/funcobject.c
- Type annotation: https://www.youtube.com/watch?v=2wDvzy6Hgxg
- Float point number: https://en.wikipedia.org/wiki/IEEE_754
- Inheritance is not subtyping: https://dl.acm.org/citation.cfm?id=96721